## Stream Processing Arquitectura Completa

## Create table with city values

In [1]:
import pandas as pd

city_list = ['Barcelona', 'Bilbao', 'Ibiza', 'Madrid', 'Oviedo', 'Sevilla', 'Valencia']
beach = [10, 0, 10, 0, 0, 0, 8]
city = [10, 9, 4, 10, 5, 7, 9]
nature = [3, 7, 9, 3, 9, 5, 6]
party = [8, 7, 10, 9, 4, 7, 8]
d = {'Beach': beach, 'City': city, 'Nature': nature, 'Party': party}

cities = pd.DataFrame(data=d, index=city_list)

cities

,Beach,City,Nature,Party
Barcelona,10,10,3,8
Bilbao,0,9,7,7
Ibiza,10,4,9,10
Madrid,0,10,3,9
Oviedo,0,5,9,4
Sevilla,0,7,5,7
Valencia,8,9,6,8


## Create streaming for Flats

In [2]:
import sys
import os
from pyspark import sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("QualityLife") \
    .getOrCreate()

Let's create a data processing for the flats

In [3]:
# Create flats_df stream
flats_df_stream = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("subscribe", "flats") \
  .load()

flats_df_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# Create schema for flats_df
schema_flats = StructType(
    [
        StructField('house_city', StringType(), True),
        StructField('house_code', StringType(), True),
        StructField('house_rent', StringType(), True),
        StructField('house_rooms', StringType(), True)
    ]
)

flats_df = flats_df_stream.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema_flats)) \
    .select(col('key'), col("timestamp"), col('value.*'))

flats_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- house_city: string (nullable = true)
 |-- house_code: string (nullable = true)
 |-- house_rent: string (nullable = true)
 |-- house_rooms: string (nullable = true)



In [6]:
# Create table to store query output in memory
flats_df.writeStream \
 .outputMode("append") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("flats_all") \
 .start()

In [8]:
spark.sql("select * from flats_all order by timestamp desc").show(truncate = False)

+---+---------+----------+----------+----------+-----------+
|key|timestamp|house_city|house_code|house_rent|house_rooms|
+---+---------+----------+----------+----------+-----------+
+---+---------+----------+----------+----------+-----------+



In [9]:
city_name = cities.index.values[0]
query = "SELECT * from flats_all WHERE house_city='{}'".format(city_name)
spark.sql(query).show()

+---+---------+----------+----------+----------+-----------+
|key|timestamp|house_city|house_code|house_rent|house_rooms|
+---+---------+----------+----------+----------+-----------+
+---+---------+----------+----------+----------+-----------+



## Create streaming for families

In [10]:
# Create families_df_stream
families_df_stream = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("subscribe", "families") \
  .load()

families_df_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [13]:
# processing_row
def processing_row(row):
    print(row[1])

In [14]:
families_df_stream.writeStream \
 .foreach(processing_row) \
 .start()

In [ ]:
families_df_query = families_df_stream.writeStream \
 .foreach(processing_row) \
 .format("console") \
 .start()

In [15]:
# Create schema for families_df
schema_families = StructType(
    [
        StructField('tweet_id', StringType(), True),
        StructField('people_name', StringType(), True),
        StructField('people_age', StringType(), True),
        StructField('people_members', StringType(), True),
        StructField('people_salary', StringType(), True),
        StructField('people_city', StringType(), True),
        StructField('people_party', StringType(), True),
        StructField('people_beach', StringType(), True),
        StructField('people_nature', StringType(), True)
    ]
)

families_df = families_df_stream.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema_families)) \
    .select(col('key'), col("timestamp"), col('value.*'))

families_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- people_name: string (nullable = true)
 |-- people_age: string (nullable = true)
 |-- people_members: string (nullable = true)
 |-- people_salary: string (nullable = true)
 |-- people_city: string (nullable = true)
 |-- people_party: string (nullable = true)
 |-- people_beach: string (nullable = true)
 |-- people_nature: string (nullable = true)



In [21]:
# processing_row
def processing_row(row):
    print(row)
    
spark.udf.register("processing_row_udf", processing_row)
    
families_df.writeStream.foreach(processing_row).start()

In [ ]:
# DEFINE FUNCTIONS FOR DATA STREAM

# best_city
def best_city(df, hobbies):
    df = abs(df - hobbies)
    df['total'] = df.sum(axis=1)
    df = df.sort_values('total')
    city = df.index.values[0]
    return city

# process_row
def process_row(row):
    # Retrieve best city
    hobbies = [row['people_beach'], row['people_city'], row['people_nature'], row['people_party']]
    city = best_city(cities, hobbies)
    
    
    return row

In [16]:
families_df_query = families_df.writeStream \
 .foreach(processing_row) \
 .outputMode("append") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("families") \
 .start()

In [25]:
spark.sql("select * from families order by timestamp desc").show(truncate = False)

+----+-----------------------+-------------------+----------------------+----------+--------------+-------------+-----------+------------+------------+-------------+
|key |timestamp              |tweet_id           |people_name           |people_age|people_members|people_salary|people_city|people_party|people_beach|people_nature|
+----+-----------------------+-------------------+----------------------+----------+--------------+-------------+-----------+------------+------------+-------------+
|null|2021-02-02 19:05:35.943|1356680159227768838|Brooke Payne          |33        |4             |125385       |4          |10          |9           |4            |
|null|2021-02-02 19:01:04.643|1356679023200514048|Lee Davis             |28        |2             |130389       |6          |10          |10          |10           |
|null|2021-02-02 18:55:33.209|1356677635561172992|Mrs. Amanda Carlson MD|58        |3             |118617       |8          |0           |4           |2            |
+---

In [ ]:
query = families_df.writeStream.format("console").start()

In [ ]:
query.lastProgress

In [ ]:
# Create table to store query output in memory
families_df.writeStream \
 .outputMode("append") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("families_all") \
 .start()

In [ ]:
spark.sql("select * from families_all order by timestamp desc").show(truncate = False)